# Hipótese 3 
Neste Jupyter Notebook, exploraremos a hipótese de que, que existe um uso pequeno do plano de saúde com serviços de apoio psicológico e bem-estar, por conta do valor pago ser um valor alto.

***
## Instalação das bibliotecas:
Instala bibliotecas necessárias:

In [ ]:
!pip install pandas numpy matplotlib seaborn

***
## Importação das bibliotecas:
Importa bibliotecas necessárias:

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#configurando estilo seaborn
sns.set(style="whitegrid")

***
## Importe do banco de dados via arquivo local
Importação do DataFrame da base de dados da Unipar com a biblioteca pandas.<br> 
Para rodar o notebook corretamente, será necessário importar o arquivo de banco de dados manualmente, pois ele não está incluído no repositório devido à presença de dados sensíveis.

#### Passos para Importar o Banco de Dados no VS Code

4. *Obtenha o Arquivo de Dados*:
   - O arquivo BASE_DE_SINISTRO_UNIPAR_BRADESCO_ORIGINAL.csv deverá ser fornecido separadamente. Entre em contato com o responsável pelo projeto para receber o arquivo.

5. *Posicione o Arquivo na Pasta Correta*:
   - Após receber o arquivo, arraste-o para a mesma pasta onde o notebook está localizado em seu computador. Isso garante que o caminho relativo na função de leitura permaneça o mesmo e funcione corretamente.
   
6. *Verifique o Caminho do Arquivo*:
   - O código de leitura do arquivo já está implementado no notebook e não precisa ser alterado:
     python
     df = pd.read_csv('BASE_DE_SINISTRO_UNIPAR_BRADESCO_ORIGINAL.csv', decimal=',')
     
   - Certifique-se de que o arquivo CSV esteja no mesmo diretório que o notebook para evitar problemas de caminho.

7. *Rodar o Notebook*:
   - Com o arquivo posicionado corretamente, execute o notebook normalmente. O pandas irá carregar os dados e você poderá seguir com a análise.

*Nota*: Caso o arquivo não esteja na mesma pasta que o notebook, o código não será capaz de localizar o banco de dados, resultando em um erro. Portanto, é essencial que o arquivo seja arrastado para o diretório correto antes da execução.

Agora que você baixou os notebooks e o banco de dados, pode seguir para os próximos passos de execução, seja localmente ou no Google Colab.

In [3]:
df = pd.read_csv('[NÃO EDITAR] BASE DE SINISTRO UNIPAR_BRADESCO_ORIGINAL.xlsx - BASE.csv', decimal=',')

***
## Pré-Processamento
Como apresentado no notebook 'Pré-Processamento', as linhas a seguir executam várias etapas para limpar e organizar o banco de dados.<br>
Os comentários no código explicam cada ação.

In [4]:
# Tratamento de valores nulos
df = df.dropna()
# Correção do ponto faltante em 'UNIPAR INDUPA DO BRASIL S.A'
df = df.replace({"UNIPAR INDUPA DO BRASIL S.A": "UNIPAR INDUPA DO BRASIL S.A."})
# Remoção de AGREGADO e DEPENDENTE
df_remove_d = df.loc[(df['Elegibilidade Sinistro'] == 'DEPENDENTE') ]
df = df.drop(df_remove_d.index)
df_remove_a = df.loc[(df['Elegibilidade Sinistro'] == 'AGREGADO') ]
df = df.drop(df_remove_a.index)
# Tratamento de valores duplicados
df = df.drop_duplicates(keep='last')
# Convertendo a coluna 'Dt Data Sinistro' para o formato 'YYYYMMDD' (Ano-Mês-Dia)
df['Dt Data Sinistro'] = pd.to_datetime(df['Dt Data Sinistro'], format='%d/%m/%Y').dt.strftime('%Y%m%d')

In [5]:
# Converter a coluna para datetime
df['Dt Data Sinistro'] = pd.to_datetime(df['Dt Data Sinistro'], format='%Y%m%d')

# Extrair componentes de ano, mês e dia
df['Ano'] = df['Dt Data Sinistro'].dt.year
df['Mes'] = df['Dt Data Sinistro'].dt.month
df['Dia'] = df['Dt Data Sinistro'].dt.day

***
## Comprovação da hipótese

A seguir, será apresentado um gráfico que mostra a distribuição das ativações de sinistros por mês, considerando os anos de 2023 e 2024. Essa análise será baseada na coluna 'Descrição Serviço Sinistro'.

In [ ]:
df['Mes_Ano'] = df['Mes'].astype(str) + '-' + df['Ano'].astype(str)
sinistros_por_mes = df.groupby('Mes')['Descricao Servico Sinistro'].count()

sinistros_por_mes = sinistros_por_mes.reset_index()

plt.figure(figsize=(10, 6))
plt.plot(sinistros_por_mes['Mes'], sinistros_por_mes['Descricao Servico Sinistro'], marker='o', linestyle='-', color='g')

plt.xticks(np.arange(1, 13, 1), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])

plt.title('Número de ativações de sinistro por mês')
plt.xlabel('Mês')
plt.ylabel('Número de ativações')
plt.grid(True)

plt.show()

Esse gráfico revela informações importantes sobre quando houve ativação do seguro ao longo do ano. 

A seguir, vamos analisar a distribuição mensal de algumas palavras-chave relacionadas a serviços de apoio psicológico e bem-estar dentro dos nossos dados. Para essa análise, criamos uma variável contendo palavras que podem estar presentes na coluna 'Descrição Serviço Sinistro', e utilizaremos essas palavras para aprofundar nossa análise.

In [7]:
keywords = [
    'consulta em psicologia', 'sessao de psicoterapia individual por psicologo', 
    'consulta ambulatorial em fisioterapia', 'sessao individual ambulatorial de fonoaudiologia', 
    'consulta/ sessao em psicologia telessaude', 'sessao de psicoterapia individual', 
    'sessao de psicoterapia infantil', 'sessao de psicoterapia individual por psicologo telessaude',
    'consulta ambulatorial por nutricionista', 'sessao individual ambulatorial em terapia ocupacional', 
    'consulta com nutricionista telessaude', 'atendimento fisiatrico no pre e pos-op', 
    'consulta/ sessao em psicologia','psico','nutri'
]

In [8]:
df_consultas = df[df['Descricao Servico Sinistro'].str.contains('|'.join(keywords), case=False, na=False)]

In [ ]:
consultas_por_mes = df_consultas.groupby('Mes')['Descricao Servico Sinistro'].count().reset_index()

plt.figure(figsize=(10, 6))
plt.scatter(consultas_por_mes['Mes'], consultas_por_mes['Descricao Servico Sinistro'], color='b')

plt.plot(consultas_por_mes['Mes'], consultas_por_mes['Descricao Servico Sinistro'], linestyle='-', color='b')

plt.title('Dispersão das ativações do plano de saúde com serviços de apoio psicológico e bem-estar')
plt.xlabel('Mês')
plt.ylabel('Número de ativações')
plt.xticks(np.arange(1, 13, 1), ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez'])
plt.grid(True)
plt.savefig('ativacoes_consultas_psico_bemestar_por_mes.png', dpi=300, bbox_inches='tight')
plt.show()

Com o gráfico das consultas de atendimento psicológico e de serviços de bem-estar nós conseguimos perceber um número de ativações do seguro bem inferior ao geral, e percebemos que não existe uma constancia nessas ativações dos funcionários, porém ela deveria ocorrer com recorrência, pois normalmente são acompanhamentos semanais. 

Para analisar se essas consultas são caras, ou seja, possuem um valor pago maior que outros atendimentos, nós vamos considerar o R$ 117,00 como o valor de referência, pois 75% da nossa base está com o valor pago no sinistro menor que R$ 117,00, então esse valor será o parametro para avaliar se os custos do plano de saúde com serviços de apoio psicológico e de bem-estar possuem um valor pago alto, ou seja, estão maiores que R$ 117,00.

In [10]:
# Selecionando somente as consultas com valor pago maior que R$ 117
df_consultas_caras = df[
    (df['Descricao Servico Sinistro'].str.contains('|'.join(keywords), case=False, na=False)) &
    (df['Valor Pago Sinistro'] > 117)
]

A seguir, vamos analisar a distribuição mensal das consultas consideradas caras, ou seja, com valor pago maior que R$ 117,00 conforme definido anteriormente.

In [ ]:
consultas_caras_por_mes = df_consultas_caras.groupby('Mes')['Descricao Servico Sinistro'].count().reset_index()

plt.figure(figsize=(10, 6))
plt.scatter(consultas_caras_por_mes['Mes'], consultas_caras_por_mes['Descricao Servico Sinistro'], color='b')

plt.plot(consultas_caras_por_mes['Mes'], consultas_caras_por_mes['Descricao Servico Sinistro'], linestyle='-', color='b')

plt.title('Dispersão das ativações do plano de saúde com serviços de apoio psicológico e bem-estar com valor pago maior que R$ 117')
plt.xlabel('Mês')
plt.ylabel('Número de ativações')
plt.xticks(np.arange(1, 13, 1), ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez'])
plt.grid(True)
plt.savefig('ativacoes_consultas_caras_psico_bemestar_por_mes.png', dpi=300, bbox_inches='tight')
plt.show()

A seguir, vamos gerar um gráfico de barras comparando o número de consultas com o número de consultas com valor maior que R$ 117,00, para entendermos melhor o quanto essas consultas caras são representativas.

In [ ]:
total_consultas_caras = df_consultas_caras['Descricao Servico Sinistro'].count()
total_consultas = df_consultas['Descricao Servico Sinistro'].count()

categorias = ['Todas as Consultas', 'Consultas Caras (Valor > R$ 117)']
valores = [total_consultas, total_consultas_caras]

plt.figure(figsize=(8, 5))
bars = plt.bar(categorias, valores, color=['#ff7f0e', '#2ca02c'])


plt.ylim(0, max(valores) + 200)

# Adicionando rótulos com valores acima das barras
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, yval + 10, f'{round(yval, 2)}', 
             ha='center', va='bottom', fontsize=12, color='black', weight='bold')

# Configurações do título e eixos
plt.title('Comparação do Número Total de Consultas e Consultas Caras', fontsize=14, weight='bold')
plt.ylabel('Número de Consultas', fontsize=12)


plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

# Salvando o gráfico com alta resolução
plt.savefig('consultas_psico_bemEstar_barras.png', dpi=300, bbox_inches='tight')

# Exibindo o gráfico
plt.show()

Com o gráfico de barras, conseguimos perceber que as consultas caracterizadas como caras representam a maior parte das consultas de apoio psicológico e bem-estar, porém para conseguirmos ter uma visualização mais clara dessa informações e verificarmos qual a porcentagem de consultas que estão com o valor pago maior que R$ 117,00.

In [ ]:
total_consultas_caras = df_consultas_caras['Descricao Servico Sinistro'].count()
total_consultas = df_consultas['Descricao Servico Sinistro'].count()

# Calculando o restante das consultas que não são "Consultas Caras"
resto_consultas = total_consultas - total_consultas_caras

valores = [total_consultas_caras, resto_consultas]
labels = ['Consultas Caras (Valor > R$ 117)', 'Outras Consultas']
colors = ['#2ca02c','#ff7f0e']

def func(pct, allvalues):
    absolute = int(pct / 100. * sum(allvalues))
    return f'{pct:.1f}%\n({absolute} consultas)'

# Criando o gráfico de pizza
fig, ax = plt.subplots(figsize=(7, 7))
wedges, texts, autotexts = ax.pie(valores, labels=labels, colors=colors, autopct=lambda pct: func(pct, valores),
                                   startangle=90, counterclock=False, wedgeprops={'edgecolor': 'black'})


for text in autotexts:
    text.set_bbox(dict(facecolor='white', edgecolor='none', pad=2))  
    text.set_color('black') 
    text.set_fontsize(10)  

for text in texts:
    text.set_fontsize(12)  

plt.title('Porcentagem de Consultas Caras em Relação ao Total de Consultas', fontsize=14, weight='bold')

# Remover a borda do gráfico
plt.gca().set_facecolor('white')

plt.savefig('consultas_psico_bemEstar_pizza.png', dpi=300, bbox_inches='tight')

# Exibindo o gráfico
plt.show()

***

# Conclusão

Com base em todos os gráficos e análises desenvolvidos, podemos concluir que, 76,4% dos serviços de apoio psicológico e bem-estar possuem os valores maiores que a maioria dos outros serviços utilizados pelos funcionários, e que isso deve impactar para que os funcionários não procurem e não mantenham a recorrência nesses serviços ao longo do ano. O gráfico de dispersão das consultas ao longo dos meses, mostra que não existe uma recorrencia dos funcionários, pois quase todo mês ele sofre uma variação significativa. Com base nessas informações, podemos concluir que a hipótese de que existe um uso pequeno do plano de saúde com serviços de apoio psicológico e bem-estar, por conta do valor pago ser um valor alto é verdadeira.